In [4]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
from transformers import T5Tokenizer, T5ForConditionalGeneration
import pandas as pd
import torch

torch.cuda.is_available()



False

In [ ]:


query = "What is the name of the technique that consists in validating authorization of the user in each API request or in each call to a protected function?"

device = 0 if torch.cuda.is_available() else -1
print("Device: ", device)

Following is a simple NER for a text

In [2]:
# grouped entity is deprecated.
ner_tagger = pipeline("ner", aggregation_strategy="simple", device=device)

output1 = ner_tagger(query)
pd.DataFrame(output1)


No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,entity_group,score,word,start,end
0,ORG,0.620003,API,96,99


In [3]:
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english").to(device)
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

# Load the NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer, device=device, grouped_entities=True)
output1 = nlp(query)
pd.DataFrame(output1)

c:\develop\github\ai-playground\src\python\langchainVarious\langchain\lib\site-packages\transformers\pipelines\token_classification.py:169: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


,entity_group,score,word,start,end
0,ORG,0.620003,API,96,99


In [6]:
# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("iarfmoose/t5-base-question-generator")
model = T5ForConditionalGeneration.from_pretrained("iarfmoose/t5-base-question-generator")

# Define the input text
answer = """
Transformer models have a maximum input sequence length that is
referred to as the maximum context size. For applications using
DistilBERT, the maximum context size is 512 tokens, which amounts
to a few paragraphs of text. As we'll see in the next section, a token
is an atomic piece of text; for now, we0ll treat a token as a single
word. We can get a rough estimate of tweet lengths per emotion by
looking at the distribution of words per tweet:
"""
context = "Machine learning and Natural Language processing and huggingface."

# Concatenate the answer and context
input_text = f"<answer> {answer} <context> {context}"

# Encode the input sequence
input_ids = tokenizer.encode(input_text, return_tensors="pt")

token_length = len(tokenizer.tokenize(input_text))
print("Token length: ", token_length)
# Generate questions
output_ids = model.generate(input_ids)

# Decode and print the questions
questions = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print(questions)


Token length:  122
How many words per tweet??


Now I want to generate more than one question, I need to use beam

In [12]:
# Load the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("iarfmoose/t5-base-question-generator")
model = T5ForConditionalGeneration.from_pretrained("iarfmoose/t5-base-question-generator")

# Define the input text
answer = """
Complete mediation is a security principle that requires every access to every object to be authorized. 
It means that access should be checked not only when a file is opened but also on each subsequent read or write to that file. 
This principle helps ensure that all accesses to objects are checked to ensure they are allowed. 
For example, whenever a subject attempts to read an object, the operating system should mediate the action by
 determining if the subject is allowed to read the object and providing the resources for the read to occur.
"""
answer = answer.replace('\n', '')  # remove all carriage returns
context = "Computer security."

# Concatenate the answer and context
input_text = f"<answer> {answer} <context> {context}"

# Encode the input sequence
input_ids = tokenizer.encode(input_text, return_tensors="pt")

token_length = len(tokenizer.tokenize(input_text))
print("Token length: ", token_length)
# Generate questions
output_ids = model.generate(input_ids, num_beams=3, num_return_sequences=3)

# Decode and print the questions
questions = []
for output in output_ids:
    question = tokenizer.decode(output, skip_special_tokens=True)
    questions.append(question)
print(questions)


Token length:  114
['What is a complete mediation principle??', 'What is a complete mediation principle??????', 'What is a complete mediation principle???????']
